In [ ]:
import os
import sys
import traceback
from pathlib import Path
import tempfile
import uuid
from datetime import datetime
from flask import Flask, request, jsonify, render_template, send_from_directory

# Check Flask installation
try:
    from flask import Flask, request, jsonify, render_template, send_from_directory
except ImportError:
    print("Flask is not installed. Please install it with: pip install flask")
    sys.exit(1)

# Check moviepy installation
try:
    import moviepy
except ImportError:
    print("MoviePy is not installed. Please install it with: pip install moviepy")
    sys.exit(1)

# Check spacy installation
try:
    import spacy
    try:
        nlp = spacy.load("en_core_web_sm")
    except OSError:
        print("spaCy model 'en_core_web_sm' is not installed. Please install it with: python -m spacy download en_core_web_sm")
        sys.exit(1)
except ImportError:
    print("spaCy is not installed. Please install it with: pip install spacy")
    sys.exit(1)

# Initialize Flask app
app = Flask(__name__)

# Set up logging to console
import logging
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
app.logger.addHandler(handler)
app.logger.setLevel(logging.DEBUG)

# Update these paths to your video directories - use Path objects for cross-platform compatibility
ISL_VIDEOS_PATH = Path(r"N:\CAI\hackathon\Olabs_Final\Sign\words")
ALPHABET_VIDEOS_PATH = Path(r"N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips")

app.logger.info(f"ISL_VIDEOS_PATH: {ISL_VIDEOS_PATH}, exists: {ISL_VIDEOS_PATH.exists()}")
app.logger.info(f"ALPHABET_VIDEOS_PATH: {ALPHABET_VIDEOS_PATH}, exists: {ALPHABET_VIDEOS_PATH.exists()}")

# Create a temporary directory for output videos
TEMP_DIR = Path(tempfile.gettempdir()) / "isl_videos"
TEMP_DIR.mkdir(exist_ok=True)
app.logger.info(f"Temporary directory created at: {TEMP_DIR}")

def rule_based_srl(sentence):
    """
    Extract key words for ISL mapping using dependency parsing.
    Returns a list of lemmatized, lowercase words.
    """
    try:
        doc = nlp(sentence)
        words = []
        for token in doc:
            if token.is_alpha and token.dep_ not in ("det", "aux", "punct"):
                words.append(token.lemma_.lower())
        if not words:
            words = [token.lemma_.lower() for token in doc if token.is_alpha]
        return words
    except Exception as e:
        app.logger.error(f"Error in rule_based_srl: {e}")
        return [word.lower() for word in sentence.split() if word.isalpha()]

def get_isl_clip(word):
    """Get video clip for a word if it exists."""
    try:
        video_path = ISL_VIDEOS_PATH / f"{word}.mp4"
        if video_path.exists():
            return moviepy.VideoFileClip(str(video_path))
        else:
            app.logger.info(f"Missing ISL clip for: {word}")
            return None
    except Exception as e:
        app.logger.error(f"Error getting clip for word '{word}': {e}")
        return None

def get_alphabet_clip(letter):
    """Get video clip for an alphabet letter if it exists."""
    try:
        letter = letter.upper()
        video_path = ALPHABET_VIDEOS_PATH / f"{letter}.mp4"
        if video_path.exists():
            return moviepy.VideoFileClip(str(video_path))
        else:
            app.logger.info(f"Missing alphabet clip for: {letter}")
            return None
    except Exception as e:
        app.logger.error(f"Error getting clip for letter '{letter}': {e}")
        return None

def spell_word_with_alphabet(word):
    """Break a word into letters and get clips for each letter."""
    clips = []
    for letter in word.lower():
        if letter.isalpha():
            clip = get_alphabet_clip(letter)
            if clip:
                clips.append(clip)
    return clips

def find_isl_clips(words):
    """Find video clips for words or spell them with alphabet clips."""
    all_clips = []
    word_status = []
    
    for word in words:
        clip = get_isl_clip(word)
        if clip:
            all_clips.append(clip)
            word_status.append({"word": word, "found": True, "spelled": False})
        else:
            # If word not found, spell it using alphabet signs
            alphabet_clips = spell_word_with_alphabet(word)
            if alphabet_clips:
                all_clips.extend(alphabet_clips)
                word_status.append({"word": word, "found": False, "spelled": True})
            else:
                word_status.append({"word": word, "found": False, "spelled": False})
    
    return all_clips, word_status

def create_isl_video(sentence):
    """Convert sentence to ISL video."""
    try:
        isl_words = rule_based_srl(sentence)
        app.logger.info(f"Extracted words: {isl_words}")
        
        clips, word_status = find_isl_clips(isl_words)
        
        if not clips:
            app.logger.warning("No clips found for any words")
            return None, word_status
        
        # Create output video
        output_filename = f"isl_{uuid.uuid4().hex}.mp4"
        output_path = str(TEMP_DIR / output_filename)
        
        try:
            app.logger.info(f"Concatenating {len(clips)} clips")
            final_video = moviepy.concatenate_videoclips(clips, method="compose")
            app.logger.info(f"Writing video to {output_path}")
            final_video.write_videofile(output_path, codec="libx264", fps=24, logger=None)
            app.logger.info("Video created successfully")
            return output_path, word_status
        except Exception as e:
            app.logger.error(f"Error creating video: {str(e)}")
            app.logger.error(traceback.format_exc())
            return None, word_status
        finally:
            # Release resources
            for clip in clips:
                try:
                    clip.close()
                except:
                    pass
    except Exception as e:
        app.logger.error(f"Error in create_isl_video: {str(e)}")
        app.logger.error(traceback.format_exc())
        return None, []

@app.route('/')
def index():
    """Render the main HTML page"""
    return render_template("index.html")

@app.route('/convert', methods=['POST'])
def convert_text():
    """API endpoint to convert text to sign language video"""
    try:
        app.logger.info("Received conversion request")
        data = request.get_json()
        
        if not data or 'text' not in data:
            return jsonify({'error': 'No text provided'}), 400
        
        text = data['text'].strip()
        if not text:
            return jsonify({'error': 'Empty text provided'}), 400
        
        app.logger.info(f"Converting text: '{text}'")
        video_path, word_status = create_isl_video(text)
        
        if not video_path:
            return jsonify({
                'error': 'Could not create sign language video',
                'word_status': word_status
            }), 500
        
        # Generate URL for the video
        video_url = f"/video/{os.path.basename(video_path)}"
        
        return jsonify({
            'success': True,
            'video_url': video_url,
            'word_status': word_status
        })
    except Exception as e:
        app.logger.error(f"Error in convert_text: {str(e)}")
        app.logger.error(traceback.format_exc())
        return jsonify({'error': f"Server error: {str(e)}"}), 500

@app.route('/video/<filename>')
def serve_video(filename):
    """Serve the generated video file"""
    try:
        return send_from_directory(TEMP_DIR, filename)
    except Exception as e:
        app.logger.error(f"Error serving video {filename}: {str(e)}")
        return "Video not found", 404

if __name__ == "__main__":
    app.logger.info("Starting Flask application...")
    try:
        app.run(host='127.0.0.1', port=5000)
    except Exception as e:
        print(f"Error starting Flask app: {str(e)}")
        traceback.print_exc()


[2025-03-01 14:52:16,811] INFO in 3493162298: ISL_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words, exists: False


ISL_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words, exists: False


[2025-03-01 14:52:16,812] INFO in 3493162298: ALPHABET_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips, exists: False


ALPHABET_VIDEOS_PATH: N:\CAI\hackathon\Olabs_Final\Sign\words\alphabet_clips, exists: False


[2025-03-01 14:52:16,814] INFO in 3493162298: Temporary directory created at: C:\Users\MATHAN\AppData\Local\Temp\isl_videos


Temporary directory created at: C:\Users\MATHAN\AppData\Local\Temp\isl_videos


[2025-03-01 14:52:16,817] INFO in 3493162298: Starting Flask application...


Starting Flask application...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Mar/2025 14:53:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Mar/2025 14:53:37] "GET /hybridaction/zybTrackerStatisticsAction?data={}&__callback__=__cb__zybTrackerStatisticsAction__0fdb3177_0981_4340_860b_37d17a3c9951__0 HTTP/1.1" 404 -
127.0.0.1 - - [01/Mar/2025 14:53:37] "GET /hybridaction/zybTrackerStatisticsAction?data={}&__callback__=__cb__zybTrackerStatisticsAction__247e2a05_66f5_46bc_a37a_ad1cab2b5dee__1 HTTP/1.1" 404 -
127.0.0.1 - - [01/Mar/2025 14:53:38] "GET /hybridaction/zybTrackerStatisticsAction?data={}&__callback__=__cb__zybTrackerStatisticsAction__249f0e58_7c95_496e_816d_954695f72c6e__2 HTTP/1.1" 404 -
127.0.0.1 - - [01/Mar/2025 14:53:38] "GET /hybridaction/zybTrackerStatisticsAction?data={}&__callback__=__cb__zybTrackerStatisticsAction__fad78e78_34d2_493b_83a2_c6a44c47a77b__3 HTTP/1.1" 404 -
127.0.0.1 - - [01/Mar/2025 14:53:38] "GET /hybridaction/zybTrackerStatisticsAction?data={}&__callback__=__c